In [1]:
import os
import gc
import warnings
warnings.filterwarnings('ignore')
import random
import scipy as sp
import numpy as np
import pandas as pd
import joblib
import itertools
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from itertools import combinations
import tensorflow as tf
from catboost import CatBoostClassifier

In [2]:
# ====================================================
# Read data
# ====================================================
def read_data():
    input_dir = ''
    train = pd.read_parquet(input_dir + 'train_df_final.parquet')
    test = pd.read_parquet(input_dir + 'test_df_final.parquet')
    return train, test

In [3]:
# ====================================================
# Amex metric
# ====================================================
def amex_metric(y_true, y_pred):
    labels = np.transpose(np.array([y_true, y_pred]))
    labels = labels[labels[:, 1].argsort()[::-1]]
    weights = np.where(labels[:,0]==0, 20, 1)
    cut_vals = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])
    gini = [0,0]
    for i in [1,0]:
        labels = np.transpose(np.array([y_true, y_pred]))
        labels = labels[labels[:, i].argsort()[::-1]]
        weight = np.where(labels[:,0]==0, 20, 1)
        weight_random = np.cumsum(weight / np.sum(weight))
        total_pos = np.sum(labels[:, 0] *  weight)
        cum_pos_found = np.cumsum(labels[:, 0] * weight)
        lorentz = cum_pos_found / total_pos
        gini[i] = np.sum((lorentz - weight_random) * weight)
    return 0.5 * (gini[1]/gini[0] + top_four)


In [4]:
# ====================================================
# tf amex metric
# ====================================================
def tf_amex_metric(y_pred, y_true):
    #y_true = y_true.get_label()
    return 'amex_metric', amex_metric(y_true, y_pred), True

In [11]:
# ====================================================
# Training
# ====================================================
def train_(train, test):
    # Label encode categorical features
    cat_features = [
        "B_30",
        "B_38",
        "D_114",
        "D_116",
        "D_117",
        "D_120",
        "D_126",
        "D_63",
        "D_64",
        "D_66",
        "D_68"
    ]
    print("Encoding categorical variables")
    cat_features = [f"{cf}_last" for cf in cat_features]
    for cat_col in cat_features:
        encoder = LabelEncoder()
        train[cat_col] = encoder.fit_transform(train[cat_col])
        test[cat_col] = encoder.transform(test[cat_col])
        
    # Round last float features to 2 decimal place
    print("Rounding float features")
    num_cols = list(train.dtypes[(train.dtypes == 'float32') | (train.dtypes == 'float64')].index)
    num_cols = [col for col in num_cols if 'last' in col]
    for col in num_cols:
        train[col + '_round2'] = train[col].round(2)
        test[col + '_round2'] = test[col].round(2)
    
    # Filling NaN
    print("printing NaN values: ")
    print(train.isna().sum())
    print(test.isna().sum())
    
    train = train.replace(np.nan, 0)
    test = test.replace(np.nan, 0)
    
    print("After the fillna")
    print(train.isna().sum())
    print(test.isna().sum())
        
    # Get feature list
    print("Getting feature list")
    features = [col for col in train.columns if col not in ['customer_ID', "target"]]
    
    
    print("KFOLD Algorithm")
    kfold = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42)
    for fold, (trn_ind, val_ind) in enumerate(kfold.split(train, train["target"])):
        X_train, X_test = train[features].iloc[trn_ind], train[features].iloc[val_ind]
        y_train, y_test = train["target"].iloc[trn_ind], train["target"].iloc[val_ind]
        
    print("Creating model")
    
    model = CatBoostClassifier(iterations=5000, random_state=42, cat_features = cat_features)
    
    
    print("\n\nFitting model")
    model.fit(X_train, y_train)
    
    return model, X_test, y_test, test, features

In [15]:
def evaluate():
    preds = model.predict(X_test)
    
    print("Calculating metric....")
    #preds = preds.squeeze(axis = 1)
    metric = tf_amex_metric(preds, y_test)
    print(metric)

In [18]:
# ====================================================
# Predict
# ====================================================
def predict(test):
    predictions = model.predict(test[features])
    
    sub_df = pd.DataFrame({"customer_ID": test["customer_ID"], "prediction":predictions})
    sub_df.isna().sum()
    sub_df.head()
    
    return sub_df, predictions

In [8]:
# ====================================================
# to_csv
# ====================================================
def to_csv(sub_df, name):
    sub_df.to_csv(name, index = False)

In [12]:
train_df, test_df = read_data()

In [13]:
model, X_test, y_test, test, features = train_(train_df, test_df)

Encoding categorical variables
Rounding float features
printing NaN values: 
customer_ID               0
P_2_mean               2434
P_2_std                7829
P_2_min                2434
P_2_max                2434
                      ...  
D_141_last_round2      2532
D_142_last_round2    373333
D_143_last_round2      2532
D_144_last_round2         0
D_145_last_round2      2532
Length: 1096, dtype: int64
customer_ID               0
P_2_mean               4175
P_2_std                9937
P_2_min                4175
P_2_max                4175
                      ...  
D_141_last_round2      4607
D_142_last_round2    752615
D_143_last_round2      4607
D_144_last_round2         0
D_145_last_round2      4607
Length: 1095, dtype: int64
After the fillna
customer_ID          0
P_2_mean             0
P_2_std              0
P_2_min              0
P_2_max              0
                    ..
D_141_last_round2    0
D_142_last_round2    0
D_143_last_round2    0
D_144_last_round2    0
D_145_

129:	learn: 0.2354685	total: 1m 15s	remaining: 47m 14s
130:	learn: 0.2353621	total: 1m 16s	remaining: 47m 12s
131:	learn: 0.2352175	total: 1m 16s	remaining: 47m 8s
132:	learn: 0.2351007	total: 1m 17s	remaining: 47m 8s
133:	learn: 0.2349880	total: 1m 17s	remaining: 47m 6s
134:	learn: 0.2348567	total: 1m 18s	remaining: 47m 6s
135:	learn: 0.2347497	total: 1m 18s	remaining: 47m 4s
136:	learn: 0.2346303	total: 1m 19s	remaining: 47m 2s
137:	learn: 0.2344992	total: 1m 19s	remaining: 46m 58s
138:	learn: 0.2343948	total: 1m 20s	remaining: 46m 54s
139:	learn: 0.2342926	total: 1m 21s	remaining: 46m 52s
140:	learn: 0.2341433	total: 1m 21s	remaining: 46m 57s
141:	learn: 0.2340422	total: 1m 22s	remaining: 46m 55s
142:	learn: 0.2339123	total: 1m 22s	remaining: 46m 53s
143:	learn: 0.2338098	total: 1m 23s	remaining: 46m 52s
144:	learn: 0.2337166	total: 1m 23s	remaining: 46m 49s
145:	learn: 0.2336113	total: 1m 24s	remaining: 46m 50s
146:	learn: 0.2335340	total: 1m 25s	remaining: 46m 47s
147:	learn: 0.23

280:	learn: 0.2257604	total: 2m 39s	remaining: 44m 31s
281:	learn: 0.2257275	total: 2m 39s	remaining: 44m 29s
282:	learn: 0.2256946	total: 2m 40s	remaining: 44m 28s
283:	learn: 0.2256709	total: 2m 40s	remaining: 44m 26s
284:	learn: 0.2256379	total: 2m 41s	remaining: 44m 24s
285:	learn: 0.2255914	total: 2m 41s	remaining: 44m 25s
286:	learn: 0.2255422	total: 2m 42s	remaining: 44m 24s
287:	learn: 0.2255002	total: 2m 42s	remaining: 44m 23s
288:	learn: 0.2254673	total: 2m 43s	remaining: 44m 22s
289:	learn: 0.2254223	total: 2m 43s	remaining: 44m 23s
290:	learn: 0.2253791	total: 2m 44s	remaining: 44m 23s
291:	learn: 0.2253418	total: 2m 45s	remaining: 44m 23s
292:	learn: 0.2253045	total: 2m 45s	remaining: 44m 22s
293:	learn: 0.2252584	total: 2m 46s	remaining: 44m 20s
294:	learn: 0.2252191	total: 2m 46s	remaining: 44m 21s
295:	learn: 0.2251912	total: 2m 47s	remaining: 44m 19s
296:	learn: 0.2251607	total: 2m 47s	remaining: 44m 17s
297:	learn: 0.2251309	total: 2m 48s	remaining: 44m 16s
298:	learn

431:	learn: 0.2213465	total: 4m 1s	remaining: 42m 37s
432:	learn: 0.2213262	total: 4m 2s	remaining: 42m 37s
433:	learn: 0.2213059	total: 4m 2s	remaining: 42m 36s
434:	learn: 0.2212777	total: 4m 3s	remaining: 42m 36s
435:	learn: 0.2212546	total: 4m 4s	remaining: 42m 35s
436:	learn: 0.2212361	total: 4m 4s	remaining: 42m 33s
437:	learn: 0.2212118	total: 4m 5s	remaining: 42m 32s
438:	learn: 0.2211849	total: 4m 5s	remaining: 42m 32s
439:	learn: 0.2211619	total: 4m 6s	remaining: 42m 31s
440:	learn: 0.2211412	total: 4m 6s	remaining: 42m 30s
441:	learn: 0.2211134	total: 4m 7s	remaining: 42m 29s
442:	learn: 0.2210882	total: 4m 7s	remaining: 42m 28s
443:	learn: 0.2210689	total: 4m 8s	remaining: 42m 26s
444:	learn: 0.2210458	total: 4m 8s	remaining: 42m 27s
445:	learn: 0.2210198	total: 4m 9s	remaining: 42m 26s
446:	learn: 0.2209968	total: 4m 9s	remaining: 42m 25s
447:	learn: 0.2209740	total: 4m 10s	remaining: 42m 24s
448:	learn: 0.2209512	total: 4m 10s	remaining: 42m 23s
449:	learn: 0.2209243	tota

582:	learn: 0.2179228	total: 5m 24s	remaining: 41m 1s
583:	learn: 0.2179023	total: 5m 25s	remaining: 41m
584:	learn: 0.2178818	total: 5m 25s	remaining: 40m 59s
585:	learn: 0.2178629	total: 5m 26s	remaining: 40m 57s
586:	learn: 0.2178423	total: 5m 26s	remaining: 40m 56s
587:	learn: 0.2178222	total: 5m 27s	remaining: 40m 55s
588:	learn: 0.2178005	total: 5m 27s	remaining: 40m 54s
589:	learn: 0.2177772	total: 5m 28s	remaining: 40m 54s
590:	learn: 0.2177588	total: 5m 28s	remaining: 40m 54s
591:	learn: 0.2177377	total: 5m 29s	remaining: 40m 54s
592:	learn: 0.2177217	total: 5m 30s	remaining: 40m 53s
593:	learn: 0.2176994	total: 5m 30s	remaining: 40m 52s
594:	learn: 0.2176849	total: 5m 31s	remaining: 40m 51s
595:	learn: 0.2176618	total: 5m 31s	remaining: 40m 51s
596:	learn: 0.2176417	total: 5m 32s	remaining: 40m 50s
597:	learn: 0.2176165	total: 5m 32s	remaining: 40m 50s
598:	learn: 0.2175959	total: 5m 33s	remaining: 40m 50s
599:	learn: 0.2175765	total: 5m 33s	remaining: 40m 48s
600:	learn: 0.2

...

4384:	learn: 0.1761621	total: 38m 30s	remaining: 5m 24s
4385:	learn: 0.1761502	total: 38m 30s	remaining: 5m 23s
4386:	learn: 0.1761425	total: 38m 31s	remaining: 5m 23s
4387:	learn: 0.1761313	total: 38m 31s	remaining: 5m 22s
4388:	learn: 0.1761211	total: 38m 32s	remaining: 5m 22s
4389:	learn: 0.1761111	total: 38m 32s	remaining: 5m 21s
4390:	learn: 0.1761041	total: 38m 33s	remaining: 5m 20s
4391:	learn: 0.1760960	total: 38m 33s	remaining: 5m 20s
4392:	learn: 0.1760888	total: 38m 34s	remaining: 5m 19s
4393:	learn: 0.1760801	total: 38m 34s	remaining: 5m 19s
4394:	learn: 0.1760720	total: 38m 35s	remaining: 5m 18s
4395:	learn: 0.1760647	total: 38m 35s	remaining: 5m 18s
4396:	learn: 0.1760594	total: 38m 36s	remaining: 5m 17s
4397:	learn: 0.1760510	total: 38m 36s	remaining: 5m 17s
4398:	learn: 0.1760424	total: 38m 37s	remaining: 5m 16s
4399:	learn: 0.1760330	total: 38m 38s	remaining: 5m 16s
4400:	learn: 0.1760234	total: 38m 38s	remaining: 5m 15s
4401:	learn: 0.1760148	total: 38m 39s	remaining:

4532:	learn: 0.1748644	total: 39m 58s	remaining: 4m 7s
4533:	learn: 0.1748556	total: 39m 59s	remaining: 4m 6s
4534:	learn: 0.1748491	total: 40m	remaining: 4m 6s
4535:	learn: 0.1748413	total: 40m 1s	remaining: 4m 5s
4536:	learn: 0.1748320	total: 40m 1s	remaining: 4m 5s
4537:	learn: 0.1748273	total: 40m 2s	remaining: 4m 4s
4538:	learn: 0.1748177	total: 40m 3s	remaining: 4m 4s
4539:	learn: 0.1748079	total: 40m 3s	remaining: 4m 3s
4540:	learn: 0.1748017	total: 40m 4s	remaining: 4m 3s
4541:	learn: 0.1747948	total: 40m 5s	remaining: 4m 2s
4542:	learn: 0.1747877	total: 40m 5s	remaining: 4m 2s
4543:	learn: 0.1747773	total: 40m 6s	remaining: 4m 1s
4544:	learn: 0.1747672	total: 40m 7s	remaining: 4m 1s
4545:	learn: 0.1747556	total: 40m 7s	remaining: 4m
4546:	learn: 0.1747478	total: 40m 8s	remaining: 4m
4547:	learn: 0.1747405	total: 40m 8s	remaining: 3m 59s
4548:	learn: 0.1747340	total: 40m 9s	remaining: 3m 58s
4549:	learn: 0.1747254	total: 40m 9s	remaining: 3m 58s
4550:	learn: 0.1747182	total: 40

4680:	learn: 0.1735983	total: 41m 22s	remaining: 2m 49s
4681:	learn: 0.1735914	total: 41m 23s	remaining: 2m 48s
4682:	learn: 0.1735813	total: 41m 23s	remaining: 2m 48s
4683:	learn: 0.1735734	total: 41m 24s	remaining: 2m 47s
4684:	learn: 0.1735633	total: 41m 24s	remaining: 2m 47s
4685:	learn: 0.1735557	total: 41m 25s	remaining: 2m 46s
4686:	learn: 0.1735476	total: 41m 25s	remaining: 2m 46s
4687:	learn: 0.1735379	total: 41m 26s	remaining: 2m 45s
4688:	learn: 0.1735272	total: 41m 26s	remaining: 2m 45s
4689:	learn: 0.1735238	total: 41m 27s	remaining: 2m 44s
4690:	learn: 0.1735130	total: 41m 27s	remaining: 2m 43s
4691:	learn: 0.1735025	total: 41m 28s	remaining: 2m 43s
4692:	learn: 0.1734949	total: 41m 29s	remaining: 2m 42s
4693:	learn: 0.1734948	total: 41m 29s	remaining: 2m 42s
4694:	learn: 0.1734855	total: 41m 29s	remaining: 2m 41s
4695:	learn: 0.1734741	total: 41m 30s	remaining: 2m 41s
4696:	learn: 0.1734649	total: 41m 30s	remaining: 2m 40s
4697:	learn: 0.1734546	total: 41m 31s	remaining:

4828:	learn: 0.1723029	total: 42m 42s	remaining: 1m 30s
4829:	learn: 0.1722940	total: 42m 43s	remaining: 1m 30s
4830:	learn: 0.1722857	total: 42m 43s	remaining: 1m 29s
4831:	learn: 0.1722742	total: 42m 44s	remaining: 1m 29s
4832:	learn: 0.1722640	total: 42m 45s	remaining: 1m 28s
4833:	learn: 0.1722575	total: 42m 45s	remaining: 1m 28s
4834:	learn: 0.1722466	total: 42m 46s	remaining: 1m 27s
4835:	learn: 0.1722377	total: 42m 46s	remaining: 1m 27s
4836:	learn: 0.1722277	total: 42m 47s	remaining: 1m 26s
4837:	learn: 0.1722193	total: 42m 48s	remaining: 1m 26s
4838:	learn: 0.1722095	total: 42m 48s	remaining: 1m 25s
4839:	learn: 0.1721997	total: 42m 49s	remaining: 1m 24s
4840:	learn: 0.1721901	total: 42m 50s	remaining: 1m 24s
4841:	learn: 0.1721802	total: 42m 50s	remaining: 1m 23s
4842:	learn: 0.1721717	total: 42m 51s	remaining: 1m 23s
4843:	learn: 0.1721669	total: 42m 51s	remaining: 1m 22s
4844:	learn: 0.1721579	total: 42m 52s	remaining: 1m 22s
4845:	learn: 0.1721483	total: 42m 53s	remaining:

4978:	learn: 0.1710132	total: 44m 15s	remaining: 11.2s
4979:	learn: 0.1710046	total: 44m 15s	remaining: 10.7s
4980:	learn: 0.1709932	total: 44m 16s	remaining: 10.1s
4981:	learn: 0.1709931	total: 44m 16s	remaining: 9.6s
4982:	learn: 0.1709861	total: 44m 17s	remaining: 9.07s
4983:	learn: 0.1709765	total: 44m 18s	remaining: 8.54s
4984:	learn: 0.1709646	total: 44m 18s	remaining: 8s
4985:	learn: 0.1709582	total: 44m 19s	remaining: 7.47s
4986:	learn: 0.1709470	total: 44m 19s	remaining: 6.94s
4987:	learn: 0.1709382	total: 44m 20s	remaining: 6.4s
4988:	learn: 0.1709288	total: 44m 21s	remaining: 5.87s
4989:	learn: 0.1709253	total: 44m 21s	remaining: 5.34s
4990:	learn: 0.1709164	total: 44m 22s	remaining: 4.8s
4991:	learn: 0.1709105	total: 44m 22s	remaining: 4.27s
4992:	learn: 0.1709002	total: 44m 23s	remaining: 3.73s
4993:	learn: 0.1708964	total: 44m 23s	remaining: 3.2s
4994:	learn: 0.1708907	total: 44m 24s	remaining: 2.67s
4995:	learn: 0.1708819	total: 44m 25s	remaining: 2.13s
4996:	learn: 0.17

In [16]:
evaluate()

Calculating metric....
('amex_metric', 0.5888453613604528, True)


In [19]:
sub_df, predictions = predict(test)

In [20]:
test["customer_ID"].shape

(924621,)

In [21]:
to_csv(sub_df, "catboostclassifier(withALLDATA).csv")